In [6]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels

## Load DATA


### Observations


In [7]:
calcofi_obs = xr.open_dataset("../data/1_products/Calcofi_obs.zarr", engine="zarr")
calcofi_obs

<xarray.Dataset> Size: 353MB
Dimensions:    (time: 7796, latitude: 55, longitude: 103, layer: 1)
Coordinates:
  * latitude   (latitude) float64 440B 0.5 1.5 2.5 3.5 ... 51.5 52.5 53.5 54.5
  * layer      (layer) int64 8B 0
  * longitude  (longitude) float64 824B -179.5 -178.5 -177.5 ... -78.5 -77.5
  * time       (time) datetime64[ns] 62kB 1951-01-09 1951-01-10 ... 2023-01-25
Data variables:
    day        (time, latitude, longitude, layer) float32 177MB ...
    night      (time, latitude, longitude, layer) float32 177MB ...

In [8]:
calcofi_forcing = xr.open_dataset(
    "/Users/adm-lehodey/Documents/Workspace/Data/phd/FORCING/Calcofi_forcing.zarr", engine="zarr"
)
calcofi_forcing

<xarray.Dataset> Size: 3GB
Dimensions:              (time: 9436, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * time                 (time) datetime64[ns] 75kB 1998-01-01 ... 2023-11-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 1GB ...
    npp                  (time, latitude, longitude) float64 428MB ...
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 1GB ...
    zooc                 (time, latitude, longitude) float64 428MB ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

In [9]:
common_latitudes = sorted(set(calcofi_obs.latitude.data).intersection(set(calcofi_forcing.latitude.data)))
common_longitudes = sorted(set(calcofi_obs.longitude.data).intersection(set(calcofi_forcing.longitude.data)))

print(f"Latitudes communes: {common_latitudes}")
print(f"Longitudes communes: {common_longitudes}")

# Sélection des données de calcofi_forcing avec les coordonnées communes
calcofi_forcing_common = calcofi_forcing.sel(latitude=common_latitudes, longitude=common_longitudes)
calcofi_forcing_common

Latitudes communes: [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.5, 22.5, 23.5, 24.5, 25.5, 26.5, 27.5, 28.5, 29.5, 30.5, 31.5, 32.5, 33.5, 34.5, 35.5, 36.5, 37.5, 38.5, 39.5, 40.5, 41.5, 42.5, 43.5, 44.5, 45.5, 46.5, 47.5, 48.5, 49.5, 50.5, 51.5, 52.5, 53.5, 54.5]
Longitudes communes: [-179.5, -178.5, -177.5, -176.5, -175.5, -174.5, -173.5, -172.5, -171.5, -170.5, -169.5, -168.5, -167.5, -166.5, -165.5, -164.5, -163.5, -162.5, -161.5, -160.5, -159.5, -158.5, -157.5, -156.5, -155.5, -154.5, -153.5, -152.5, -151.5, -150.5, -149.5, -148.5, -147.5, -146.5, -145.5, -144.5, -143.5, -142.5, -141.5, -140.5, -139.5, -138.5, -137.5, -136.5, -135.5, -134.5, -133.5, -132.5, -131.5, -130.5, -129.5, -128.5, -127.5, -126.5, -125.5, -124.5, -123.5, -122.5, -121.5, -120.5, -119.5, -118.5, -117.5, -116.5, -115.5, -114.5, -113.5, -112.5, -111.5, -110.5, -109.5, -108.5, -107.5, -106.5, -105.5, -104.5, -103.5, -102.5, -101.5, -100.

<xarray.Dataset> Size: 3GB
Dimensions:              (time: 9436, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * time                 (time) datetime64[ns] 75kB 1998-01-01 ... 2023-11-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 1GB ...
    npp                  (time, latitude, longitude) float64 428MB ...
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 1GB ...
    zooc                 (time, latitude, longitude) float64 428MB ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Change the depth to be categorical AND starting with 0 value rather than 1


In [10]:
depth_attrs = {
    "flag_values": "[0 1 2]",
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "depth",
    "long_name": "Layer depth",
    "axis": "Z",
}
calcofi_forcing_common = calcofi_forcing_common.assign_coords(depth=[0, 1, 2])
calcofi_forcing_common.depth.attrs = depth_attrs
calcofi_forcing_common

<xarray.Dataset> Size: 3GB
Dimensions:              (time: 9436, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * time                 (time) datetime64[ns] 75kB 1998-01-01 ... 2023-11-01
  * depth                (depth) int64 24B 0 1 2
Data variables:
    T                    (time, depth, latitude, longitude) float64 1GB ...
    npp                  (time, latitude, longitude) float64 428MB ...
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 1GB ...
    zooc                 (time, latitude, longitude) float64 428MB ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Gathering Data & Save


In [11]:
calcofi_forcing_common.to_zarr("./products/Calcofi_cmems.zarr", mode="w")